In [1]:
%matplotlib widget

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for pd in physical_devices:
    print(pd)
    
    tf.config.experimental.set_memory_growth(pd, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')


In [4]:
mnist = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# x_train = x_train[..., tf.newaxis].astype("float32")
# x_test = x_test[..., tf.newaxis].astype("float32")

In [5]:
class FT_Kernel(tf.keras.layers.Layer):
    def __init__(self, num_outputs, num_freq, dims):
        super(FT_Kernel, self).__init__()
        self.outputs     = tf.range(num_outputs)
        self.frequencies = tf.constant(num_freq)
        self.dims        = tf.constant(dims)
        self.output_dim  = num_outputs*len(dims)
        
        
    
    def build(self, input_shape):
        self.im_dim = input_shape[1]
        if len(input_shape) == 3:
            input_shape = [1]
        self.amplitudes = self.add_weight("amplitudes", 
                                          shape=[int(input_shape[-1]),
                                                 self.outputs.shape[0],
                                                 2, 
                                                 self.frequencies],
                                          initializer='random_normal',
                                          trainable=True)
    @tf.function
    def get_kernels(self, dim):
        
        print("Running GK against dim", dim)

            
        transformed_freq = tf.cast(self.frequencies+1, tf.float32)

        basis_linspace = tf.expand_dims(tf.linspace(0.0, np.pi, dim), -1)
        # print("BL Shape")
        # print(basis_linspace.shape)

        # basis_multi = tf.broadcast_to(basis_linspace, [dim, dim])
        basis_multi = tf.tile(input=basis_linspace, multiples=[1, dim])
        # print("BM Shape")
        # print(basis_multi.shape)


        basis_transformed = basis_multi * tf.transpose(transformed_freq)
        # print("BT Shape")
        # print(basis_transformed.shape)


        oscillations = tf.sin(basis_transformed)
        # print("Osc shape")
        # print(oscillations.shape)

        # print("Single_freq_stack shape")
        # print(self.amplitudes[:, :, 0].shape)

        
        # Creating oscillations_multi
        pre_om_1 = tf.reshape(oscillations, [dim, dim, 1, 1, 1])
        oscillations_multi = tf.tile(pre_om_1, [1, 1, self.amplitudes.shape[0], self.amplitudes.shape[1], self.amplitudes.shape[3]])
        # oscillations_multi = tf.broadcast_to(oscillations, [dim, dim, self.amplitudes.shape[0], self.amplitudes.shape[1], self.amplitudes.shape[3]])
        # print("OM Shape")
        # print(oscillations_multi.shape)
        
        
        # Creating amp_multi_h
        pre_amh = tf.reshape(self.amplitudes[:, :, 0], [1, 1, self.amplitudes.shape[0], self.amplitudes.shape[1], self.amplitudes.shape[3]])
        amp_multi_h = tf.tile(pre_amh, [dim, dim, 1, 1, 1])
        # amp_multi_h = tf.broadcast_to(self.amplitudes[:, :, 0], [dim, dim, self.amplitudes.shape[0], self.amplitudes.shape[1], self.amplitudes.shape[3]])
        
        
        
        # Creating amp_multi_v
        pre_amv = tf.reshape(self.amplitudes[:, :, 1], [1, 1, self.amplitudes.shape[0], self.amplitudes.shape[1], self.amplitudes.shape[3]])
        amp_multi_v = tf.tile(pre_amv, [dim, dim, 1, 1, 1])
        # amp_multi_v = tf.broadcast_to(self.amplitudes[:, :, 1], [dim, dim, self.amplitudes.shape[0], self.amplitudes.shape[1], self.amplitudes.shape[3]])
        
        
        # print("OA Multi")
        # print(oscillations_multi.shape)
        # print(amp_multi_h.shape)

        h_contrib = oscillations_multi*amp_multi_h
        v_contrib = tf.transpose(oscillations_multi*amp_multi_v, perm=[1, 0, 2, 3, 4])

        h_contrib = tf.reduce_sum(h_contrib, axis=-1)
        v_contrib = tf.reduce_sum(v_contrib, axis=-1)
        # print("HV Shapes")
        # print(h_contrib.shape)
        # print(v_contrib.shape)

        kernel = h_contrib + v_contrib

        return kernel
    
    @tf.function
    def call(self, input_tensor):
        # print("Current Amp Shape: ", self.amplitudes.shape)
        # print(input_tensor.shape)
        rs_input_tensor = tf.reshape(input_tensor, [-1, self.im_dim, self.im_dim, self.amplitudes.shape[0]])
        # Prep Result Storage

        def get_dim_result(dim):
            kernel = self.get_kernels(dim)
            dim_result = tf.nn.conv2d(rs_input_tensor, kernel, strides=[1,1,1,1], padding='SAME')
            # print("Dim Result")
            # print(dim_result.shape)
            return dim_result
        
        # output_tensor = tf.scan(get_dim_result, tf.expand_dims(self.dims, 1)) # Incorrect
        output_tensor = tf.map_fn(get_dim_result, self.dims, dtype=tf.float32)
        # output_tensor = tf.stack([get_dim_result(dim) for dim in self.dims])
        
        output_tensor = tf.reduce_mean(output_tensor, axis=0)
        print("OT shape")
        print(output_tensor.shape)
        
        # output_tensor = tf.reshape(output_tensor, shape=[-1, rs_input_tensor.shape[1], rs_input_tensor.shape[2], self.output_dim])
        



        
        return output_tensor
                    
                    
                    
        

In [6]:
model = tf.keras.models.Sequential([
        FT_Kernel(32, 16, [5, 7]),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

In [7]:
predictions = model(x_train[:1]).numpy()
print(predictions)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(1, 32, 32, 32)
[[-0.8816831   1.795973   -0.34890792 -0.4725381   0.15341452  1.991337
  -0.43258393  4.326099   -1.6111448   1.0806417 ]]


In [8]:
tf.nn.softmax(predictions).numpy()

array([[0.00432677, 0.06295861, 0.00737132, 0.00651408, 0.01218151,
        0.07654215, 0.00677962, 0.790451  , 0.00208623, 0.03078866]],
      dtype=float32)

# Investigate Gradients
$\frac{11}{22x}\neq 6$

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
loss_fn(y_train[:1], predictions).numpy()

4.9938345

In [11]:
with tf.GradientTape() as tape:
    tape.watch(model.layers[0].amplitudes)
    predictions=model(x_train[:50])
    loss=loss_fn(y_train[:50], predictions)

grads = tape.gradient(loss, model.layers[0].amplitudes)
print(grads.numpy())

OT shape
(50, 32, 32, 32)
[[[[-0.25951546 -0.25951546 -0.25951546 ... -0.25951546 -0.25951546
    -0.25951546]
   [-0.26767516 -0.26767516 -0.26767516 ... -0.26767516 -0.26767516
    -0.26767516]]

  [[ 1.1164415   1.1164415   1.1164415  ...  1.1164415   1.1164415
     1.1164415 ]
   [ 1.1669052   1.1669052   1.1669052  ...  1.1669052   1.1669052
     1.1669052 ]]

  [[-0.5563036  -0.5563036  -0.5563036  ... -0.5563036  -0.5563036
    -0.5563036 ]
   [-0.5381623  -0.5381623  -0.5381623  ... -0.5381623  -0.5381623
    -0.5381623 ]]

  ...

  [[-0.6490402  -0.6490402  -0.6490402  ... -0.6490402  -0.6490402
    -0.6490402 ]
   [-0.7338837  -0.7338837  -0.7338837  ... -0.7338837  -0.7338837
    -0.7338837 ]]

  [[-0.2580031  -0.2580031  -0.2580031  ... -0.2580031  -0.2580031
    -0.2580031 ]
   [-0.4168073  -0.4168073  -0.4168073  ... -0.4168073  -0.4168073
    -0.4168073 ]]

  [[ 0.36589214  0.36589214  0.36589214 ...  0.36589214  0.36589214
     0.36589214]
   [ 0.4026953   0.4026953   0

In [12]:
print(np.mean(grads.numpy()))

-0.002806225


# Training

In [13]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # test_kernel = 
    # test_kernel = tf.keras.layers.Conv2D(142, 5)
    model2 = tf.keras.models.Sequential([
        FT_Kernel(128, 28, [3, 5, 7]),
        FT_Kernel(128, 14, [3, 5, 7]),
        FT_Kernel(128, 14, [3, 5, 7, 9]),
        
#         tf.keras.layers.Conv2D(142, 5),
#         tf.keras.layers.Conv2D(142, 5),
#         tf.keras.layers.Conv2D(142, 5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])
    model2.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])
    model2.fit(x_train, y_train, epochs=1)

Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(None, 32, 32, 128)
Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(None, 32, 32, 128)
Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(None, 32, 32, 128)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Train on 50000 samples
Running GK against dim Tensor("dim:0", shape=(), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
OT shape
(None, 32, 32, 128)
Running GK against dim Tensor("dim:0", shape=(), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
OT shape
(None, 32, 32, 128)
Running GK against dim Tensor("dim:0", shape=(), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
OT shape
(None, 32, 3

In [14]:
predictions = model2(x_train[:1]).numpy()
print(predictions)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(1, 32, 32, 128)
Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(1, 32, 32, 128)
Running GK against dim Tensor("dim:0", shape=(), dtype=int32)
OT shape
(1, 32, 32, 128)
[[-0.00803585  0.00433486 -0.00244662  0.01380249  0.00610123 -0.01876303
   0.00854035  0.01058773  0.00913703  0.00609751]]


In [15]:
tf.nn.softmax(predictions).numpy()

array([[0.09890448, 0.10013559, 0.09945882, 0.10108814, 0.10031263,
        0.09784918, 0.1005576 , 0.10076369, 0.10061762, 0.10031225]],
      dtype=float32)

In [16]:
weights = model2.trainable_weights

In [17]:
print(weights)

[MirroredVariable:{
  0 /job:localhost/replica:0/task:0/device:GPU:0: <tf.Variable 'sequential_1/ft__kernel_1/amplitudes:0' shape=(3, 128, 2, 28) dtype=float32, numpy=
array([[[[-0.0580604 , -0.00538922, -0.04349372, ...,  0.02351996,
           0.0190387 , -0.01704978],
         [-0.02994329,  0.04227976,  0.03223782, ..., -0.01215796,
           0.03395325, -0.05799717]],

        [[-0.04445405,  0.08062734, -0.04771588, ...,  0.07421277,
           0.03449752,  0.02824093],
         [-0.01489173, -0.04064555, -0.03422705, ..., -0.06077602,
           0.02290673,  0.05228181]],

        [[ 0.03888537,  0.04210606,  0.02761531, ..., -0.01175651,
           0.04203332, -0.09712245],
         [-0.0028976 , -0.06917026,  0.04639854, ..., -0.03471576,
          -0.09885179,  0.05098503]],

        ...,

        [[ 0.00492249,  0.06843267, -0.01164086, ..., -0.02761945,
           0.07842953,  0.00895208],
         [ 0.02247304,  0.06959274, -0.03006667, ...,  0.01362891,
           0.0243

In [18]:
mk_5 = test_kernel.get_kernels(5)

NameError: name 'test_kernel' is not defined

In [ ]:
fig1, ax1 = plt.subplots()

In [ ]:
mk_5_sample = mk_5[:, :, 0, 0]
print(mk_5_sample)
ax1.matshow(mk_5_sample)